In [27]:

from backdoor.defense import FoolsGold
import torch

name = "foolsgold"
memory_size = 0

defender = FoolsGold(name)
state_dict_list = torch.load("/root/autodl-tmp/GitRepos/BackdoorFIT/output/natural_instruction_20000_fedavg_c5s5_i40_b4a1_l1024_r128a256_pTruenbadnetspcr0.1pr0.1_2024-05-20_11-30-38/locals/local_dict_list_30.pth")
local_state_list = state_dict_list[:-1]
global_state = state_dict_list[-1]


In [26]:
from utils import flatten_dict
import numpy as np 

key_order = global_state.keys()
total_params = sum(p.numel() for p in global_state.values())
clients_this_round = list(range(len(local_state_list)))

flatten_global_model = flatten_dict(global_state, key_order)
delta = np.zeros((len(local_state_list), total_params))
summed_deltas = np.zeros((len(local_state_list), total_params))

for client_idx in range(len(local_state_list)):
    flatten_local_model = flatten_dict(local_state_list[client_idx], key_order)
    local_update = flatten_local_model - flatten_global_model
    local_update = local_update.detach().cpu().numpy()
    delta[client_idx,:] = local_update
# normalize delta
    if np.linalg.norm(delta[client_idx, :]) > 1:
        delta[client_idx, :] = delta[client_idx, :] / np.linalg.norm(delta[client_idx, :])

    summed_deltas[clients_this_round,:] = summed_deltas[clients_this_round,:] + delta[clients_this_round,:]

n_freq = defender(delta[clients_this_round,:], summed_deltas[clients_this_round, :], global_state, round, "0", len(local_state_list), total_params, key_order)
print(n_freq)


START Aggregating history of gradient directions
wv: [0.34245991 0.71251132 0.34245991 0.72634877 1.        ]
[0.10962997 0.22809268 0.10962997 0.23252239 0.32012498]


In [28]:
from backdoor.defense import FoolsGold
import torch

name = "foolsgold"
memory_size = 0

rounds = [5, 10, 15, 20, 25, 30, 35]
defender = FoolsGold(name)

for round in rounds:

    state_dict_list = torch.load(f"/root/autodl-tmp/GitRepos/BackdoorFIT/output/natural_instruction_20000_fedavg_c5s5_i40_b4a1_l1024_r128a256_pTruenbadnetspcr0.1pr0.1_2024-05-20_11-30-38/locals/local_dict_list_{round}.pth")
    local_state_list = state_dict_list[:-1]
    global_state = state_dict_list[-1]
    
    
    from utils import flatten_dict
    import numpy as np 

    key_order = global_state.keys()
    total_params = sum(p.numel() for p in global_state.values())
    clients_this_round = list(range(len(local_state_list)))

    flatten_global_model = flatten_dict(global_state, key_order)
    delta = np.zeros((len(local_state_list), total_params))
    summed_deltas = np.zeros((len(local_state_list), total_params))

    for client_idx in range(len(local_state_list)):
        flatten_local_model = flatten_dict(local_state_list[client_idx], key_order)
        local_update = flatten_local_model - flatten_global_model
        local_update = local_update.detach().cpu().numpy()
        delta[client_idx,:] = local_update
# normalize delta
        if np.linalg.norm(delta[client_idx, :]) > 1:
            delta[client_idx, :] = delta[client_idx, :] / np.linalg.norm(delta[client_idx, :])

        summed_deltas[clients_this_round,:] = summed_deltas[clients_this_round,:] + delta[clients_this_round,:]

    n_freq = defender(delta[clients_this_round,:], summed_deltas[clients_this_round, :], global_state, round, "0", len(local_state_list), total_params, key_order)
    print(round, n_freq)

START Aggregating history of gradient directions
wv: [0.50874185 1.         0.50874185 0.54659196 0.54659196]
5 [0.16354748 0.3214744  0.16354748 0.17571532 0.17571532]
START Aggregating history of gradient directions
wv: [0.59609011 1.         0.57768263 0.59609011 0.57768263]
10 [0.17806782 0.29872634 0.17256902 0.17806782 0.17256902]
START Aggregating history of gradient directions
wv: [0.34245991 0.71251132 0.34245991 0.72634877 1.        ]
15 [0.10962997 0.22809268 0.10962997 0.23252239 0.32012498]
START Aggregating history of gradient directions
wv: [0.5477578 0.5449403 1.        0.5477578 0.5449403]
20 [0.17195908 0.17107458 0.31393269 0.17195908 0.17107458]
START Aggregating history of gradient directions
wv: [0.37817432 0.05710702 1.         0.37817432 0.05710702]
25 [0.20217142 0.03052933 0.5345985  0.20217142 0.03052933]
START Aggregating history of gradient directions
wv: [0.6627964  0.23792905 0.23792905 1.         0.6627964 ]
30 [0.2365904  0.08493065 0.08493065 0.3569578

In [23]:
import sklearn.metrics.pairwise as smp
cs = smp.cosine_similarity(summed_deltas) - np.eye(len(local_state_list))

In [24]:
cs[cs<0] = 0
cs[cs>1] = 1
for i in range(5):
    print( cs[i, 0], cs[i, 1],cs[i, 2],cs[i, 3],cs[i, 4])

2.4225066397320916e-13 0.00032732961575713973 0.44669911814154023 0.4759983418993628 0.3949878837597612
0.00032732961575713973 0.0 0.0002412932601735299 0.0003475241320809415 0.0003179499799409736
0.44669911814154023 0.0002412932601735299 0.0 0.44443827802898817 0.48059154841785146
0.4759983418993628 0.0003475241320809415 0.44443827802898817 3.752553823233029e-14 0.4099465358149259
0.3949878837597612 0.0003179499799409736 0.48059154841785146 0.4099465358149259 0.0


In [30]:
data_file = "/home/zx/nas/GitRepos/BackdoorFIT/data/natural-instructions/train/0.json"



0 2
1 1
2 1
3 1
4 1
5 1
6 2
7 2
8 4
9 1
10 3
11 0
12 4
13 1
14 4
15 1
16 2
17 4
18 2
19 2
20 4
21 2
22 4
23 1
24 2
25 2
26 1
27 3
28 4
29 3
30 1
31 4
32 3
33 4
34 2
35 2
36 1
